In [ ]:
# region imports
from AlgorithmImports import *
# endregion

class letsgo(QCAlgorithm):

    def Initialize(self):
        
        self.SetStartDate(2022, 3, 1)  # Set Start Date
        self.SetEndDate(2022, 5, 10) # Set End Date
        self.SetCash(1000)  # Set Strategy Cash
        
        
        self.btc = self.AddCrypto("BTCUSDT", Resolution.Minute).Symbol 
        self.SetBrokerageModel(BrokerageName.BinanceFutures, AccountType.Margin)    
        self.fastTime = 48
        self.slowTime = 120
        self.SetWarmUp(timedelta(minutes = self.slowTime + 1))
        self.fastSMA = self.EMA(self.btc, self.fastTime, Resolution.Minute)
        self.fastSMAValues = RollingWindow[Decimal](2)
        self.slowSMA = self.EMA(self.btc, self.slowTime, Resolution.Minute)
        self.slowSMAValues = RollingWindow[Decimal](2)


        #chart = Chart("My Chart")
        #self.AddChart(chart)


        #self.slowSMA = self.SMA(self.btc, 3000, Resolution.Minute)
        self.entryTicket = None
        self.stopMarketTicket = None
        self.topProfitTicket = None
        self.entryTicketS = None
        self.stopMarketTicketS = None
        self.topProfitTicketS = None
        
        self.tpL = 1.8 / 100
        self.slL = 0.6 / 100
        self.tpS = 1.8 / 100
        self.slS = 0.6 / 100        
        
    def OnData(self, data):

        price = data[self.btc].Close
        self.fastSMAValues.Add(self.fastSMA.Current.Value)
        self.slowSMAValues.Add(self.slowSMA.Current.Value)

        if self.IsWarmingUp:
            return

        if not self.fastSMAValues.IsReady or not self.slowSMAValues.IsReady:
            return


        if -50 < self.Portfolio.TotalHoldingsValue < 50:
        
            if self.fastSMAValues[1] < self.slowSMAValues[1] and self.fastSMAValues[0] > self.slowSMAValues[0]:
                
                cash_available = self.Portfolio.Cash

                quantity = cash_available * 0.98 / price

                self.entryTicket = self.MarketOrder(self.btc, quantity)

                self.stopMarketTicket = self.StopLimitOrder(self.btc, -self.entryTicket.QuantityFilled, (1 - self.slL) * self.entryTicket.AverageFillPrice, (0.999 - self.slL) * self.entryTicket.AverageFillPrice, "Stop Market")
                    
                self.topProfitTicket = self.LimitOrder(self.btc, -self.entryTicket.QuantityFilled, (1 + self.tpL) * self.entryTicket.AverageFillPrice, "Top Profit Limit")
                    
            if self.fastSMAValues[1] > self.slowSMAValues[1] and self.fastSMAValues[0] < self.slowSMAValues[0]:

                cash_available = self.Portfolio.Cash

                quantity = cash_available * 0.98 / price
                self.Log(str(quantity))        
                self.entryTicketS = self.MarketOrder(self.btc, -quantity)

                self.stopMarketTicketS = self.StopLimitOrder(self.btc, -self.entryTicketS.QuantityFilled, (1 + self.slS) * self.entryTicketS.AverageFillPrice, (1.001 + self.slS) * self.entryTicketS.AverageFillPrice, "Stop Market")
                    
                self.topProfitTicketS = self.LimitOrder(self.btc, -self.entryTicketS.QuantityFilled, (1 - self.tpS) * self.entryTicketS.AverageFillPrice, "Top Profit Limit")
                

    def OnOrderEvent(self, orderEvent: OrderEvent) -> None:

        if orderEvent.Status == OrderStatus.Filled:
        
            if self.entryTicket is not None and self.entryTicket.OrderId != orderEvent.OrderId or self.entryTicketS is not None and self.entryTicketS.OrderId != orderEvent.OrderId:
                
                self.Transactions.CancelOpenOrders()